In [6]:
!pip install pulp
from pulp import LpMaximize, LpProblem, LpVariable

# Define the LP problem
model = LpProblem("Chemical_Manufacturing_Optimization", LpMaximize)

# Decision variables
P = LpVariable("Primary_Product", lowBound=0, cat="Continuous")
K = LpVariable("Product_K", lowBound=0, cat="Continuous")
M = LpVariable("Product_M", lowBound=0, cat="Continuous")
T = LpVariable("Waste_Treated", lowBound=0, cat="Continuous")

# Objective function: Maximize profit
model += (
    5.70 * P + 0.80 * K + 0.65 * M - 0.25 * T
    - 0.20 * K - 0.10 * M,
    "Total_Profit"
)

# Constraints
model += P + K <= 7500, "Material_X_Limit"
model += 2 * P + M <= 9000, "Material_Y_Limit"
model += P == K + M + T, "Waste_Balance"

# Solve the model
model.solve()

# Print results
print("Optimal Production Plan:")
print(f"Primary Product (P): {P.varValue} lbs")
print(f"Product K (K): {K.varValue} lbs")
print(f"Product M (M): {M.varValue} lbs")
print(f"Waste Treated (T): {T.varValue} lbs")
print(f"Total Profit: ${model.objective.value():,.2f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 85.1 MB/s eta 0:00:00
Optimal Production Plan:
Primary Product (P): 4500.0 lbs
Product K (K): 3000.0 lbs
Product M (M): 0.0 lbs
Waste Treated (T): 1500.0 lbs
Total Profit: $27,075.00
